<a href="https://colab.research.google.com/github/kcw0331/Environmental-Health-Big-Data-Analysis/blob/main/6%EC%A3%BC%EC%B0%A8/1-pandas_data_structures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas Data Structures Overview
In this section, we will discuss the `Series`, `Index`, and `DataFrame` classes. To do so, we will read in a snippet of the CSV file we will work with later. Don't worry about that part yet, though.

## About the Data
In this notebook, we will be working with 5 rows from the earthquake data collected over September 18, 2018 - October 13, 2018 (obtained from the US Geological Survey (USGS) using the [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Working with NumPy Arrays
Let's read in a short CSV file (using `numpy`) for some sample data. 

# 데이터가 처음에는 없기 때문에 처음에는 google drive와 연동을 해줘야 한다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
os.listdir()

['.config', 'drive', 'sample_data']

In [3]:
cd drive

/content/drive


In [4]:
os.listdir()


['MyDrive', '.file-revisions-by-id', '.shortcut-targets-by-id', '.Trash-0']

In [5]:
cd MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition


In [6]:
cd ch_02

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_02


## chaper2에 있는 데이터를 읽어 온 것이고, 아래 보이는 것을 먼저 numpy를 사용해서 데이터를 읽어온 경우이다.

In [7]:
import numpy as np

data = np.genfromtxt(
    'data/example_data.csv', delimiter=';', 
    names=True, dtype=None, encoding='UTF'
)
data

array([('2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia', 'mww', 6.7, 'green', 1),
       ('2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww', 5.2, 'green', 0),
       ('2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww', 5.7, 'green', 0),
       ('2018-10-12 21:09:49.240', '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0),
       ('2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea', 'mww', 5.6, 'green', 1)],
      dtype=[('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i8')])

We can find the dimensions with the `shape` attribute:

In [8]:
data.shape

(5,)

* data.shape을 하게 되면, 각 row가 5개로 되어 있다는 것이 나오게 된다.

We can find the data types with the `dtype` attribute:

In [9]:
data.dtype

dtype([('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i8')])

* data.dtype을 사용해서 데이터의 정보를 확인해 줄 수 있다.

Each of the entries in the array is a row from the CSV file. NumPy arrays contain a single data type (unlike lists, which allow mixed types); this allows for fast, vectorized operations. When we read in the data, we got an array of `numpy.void` objects, which are created to store flexible types. This is because NumPy has to store several different data types per row: four strings, a float, and an integer. This means we can't take advantage of the performance improvements NumPy provides for single data type objects.

Say we want to find the maximum magnitude&mdash;we can use a **[list comprehension](https://www.python.org/dev/peps/pep-0202/)** to select the third index of each row, which is represented as a `numpy.void` object. This makes a list, meaning that we can take the maximum using the `max()` function:

* 아래 코드에서는 magnitude가 최대가 되는 것을 찾고 싶을 때 사용하는 것이다.

In [10]:
max([row[3] for row in data])   

#리스트 안에서 for문을 사용해서 하는 방법이다.
#각 row의 3번째 꺼를 아이템으로 만드는 것이다.
#그리고 maximum을 사용해서 값중에서 가장 최대가 되는 값을 골라내 준다.

6.7

In [11]:
%%timeit
max([row[3] for row in data])

The slowest run took 6.06 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 4.81 µs per loop


* %%timeit을 사용해서 코드를 사용해서 나오는 시간이 출력되게 된다.

If we, instead, create a NumPy array for each column, this operation is much easier (and more efficient) to perform. We can use a **[dictionary comprehension](https://www.python.org/dev/peps/pep-0274/)** to make a dictionary where the keys are the column names and the values are NumPy arrays of the data:

### 여기에서는 위와 같은 코드 말고 dictionary comprehension을 사용해서 쉽게 할 수 있다는 것이다.
### 위에서 코드처럼 array를 했던 것을 아래에서는 dictionary형태로 지정을 해주었다.

In [12]:
array_dict = {
    col: np.array([row[i] for row in data])
    for i, col in enumerate(data.dtype.names)
}
array_dict

{'alert': array(['green', 'green', 'green', 'green', 'green'], dtype='<U5'),
 'mag': array([6.7, 5.2, 5.7, 5.7, 5.6]),
 'magType': array(['mww', 'mww', 'mww', 'mww', 'mww'], dtype='<U3'),
 'place': array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
        '42km WNW of Sola, Vanuatu',
        '13km E of Nueva Concepcion, Guatemala',
        '128km SE of Kimbe, Papua New Guinea'], dtype='<U37'),
 'time': array(['2018-10-13 11:10:23.560', '2018-10-13 04:34:15.580',
        '2018-10-13 00:13:46.220', '2018-10-12 21:09:49.240',
        '2018-10-12 02:52:03.620'], dtype='<U23'),
 'tsunami': array([1, 0, 0, 0, 1])}

* 여기에서는 row별로 저장을 해둔게 아니라 coulmn별로 저장을 해둔 것이다.

Grabbing the maximum magnitude is now simply a matter of selecting the `mag` key and calling the `max()` method. This is nearly twice as fast as the list comprehension implementation when dealing with just 5 entries, imagine how much worse the first attempt will perform on large data sets:

In [13]:
array_dict['mag']

array([6.7, 5.2, 5.7, 5.7, 5.6])

* array_dict['mag']를 하게 되면 magnitude을 값들이 나오게 된다.

In [14]:
%%timeit
array_dict['mag'].max()

The slowest run took 28.21 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 2.57 µs per loop


* 그런데 그럼에도 불고하고 magnitude가 최대가 되는 것을 모두 출력을 한다고 하면 아래와 같은 코드 형식으로 매번 코드를 작성을 해줘야 한다.

However, this representation has other issues. Say we wanted to grab all the information for the earthquake with the maximum magnitude, how would we go about that? We would need to find the index of the maximum and then for each of the keys in the dictionary grab that index:

In [15]:
np.array([
    value[array_dict['mag'].argmax()]  #magnitude을 maximum이 되는 인덱스를 넣어주게 된다. 그리고 그 인덱스를 따라서 for문을 돌게 된다.
    for key, value in array_dict.items() #그럼 그 인덱스의 모든 for문을 돌면서 출력을 해주게 된다.
])

array(['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
       'mww', '6.7', 'green', '1'], dtype='<U31')

* 이렇게 코드를 매번 작성하는게 불편하니깐 이래서 pandas가 등장을 한 것이다.

The result is now a NumPy array of strings (our numeric values were converted), and we are now in the format from earlier. Also, consider trying to sort the data by magnitude from smallest to largest. In the first representation, we would have to sort the rows by examining the 3rd index. With the second representation, we would have to determine the order for the indices from the `mag` column, and then sort all the other arrays with those same indices. Clearly, working with several NumPy arrays of different data types at once is a bit cumbersome. However, `pandas` builds on top of NumPy arrays to make this easier. Let's start our exploration of `pandas` with an overview of the data structures.

## `Series`
The `Series` class provides a data structure for arrays of a single type with some additional functionality.

### pandas에서 제공해주는 3가지 class가 있는데, 그중에서 Series class를 알아 본다. 

In [16]:
array_dict['place']

array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
       '42km WNW of Sola, Vanuatu',
       '13km E of Nueva Concepcion, Guatemala',
       '128km SE of Kimbe, Papua New Guinea'], dtype='<U37')

* 처음에는 place의 정보를 확인해 주었다. 그리고 보니깐 array로 정의되어 있다.

In [17]:
import pandas as pd

place = pd.Series(array_dict['place'], name='place')
place

0          262km NW of Ozernovskiy, Russia
1              25km E of Bitung, Indonesia
2                42km WNW of Sola, Vanuatu
3    13km E of Nueva Concepcion, Guatemala
4      128km SE of Kimbe, Papua New Guinea
Name: place, dtype: object

* place를 pandas로 지정했을 때, 나오는 format이다.

In [18]:
place.values

array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
       '42km WNW of Sola, Vanuatu',
       '13km E of Nueva Concepcion, Guatemala',
       '128km SE of Kimbe, Papua New Guinea'], dtype=object)

In [19]:
place.index

RangeIndex(start=0, stop=5, step=1)

* Series의 Attribute를 접근해 볼려면, place.values라든가 place,name을 하게 되면 된다.

Here are some commonly used attributes with `Series` objects:

|Attribute | Returns |
| --- | --- |
| `name` | The name of the `Series` object |
| `dtype` | The data type of the `Series` object |
| `shape` | Dimensions of the `Series` object in a tuple of the form `(number of rows,)` |
| `index` | The `Index` object that is part of the `Series` object |
| `values` | The data in the `Series` object |

For the most part, `pandas` objects use NumPy arrays for their internal data representations. However, for some data types, `pandas` builds upon NumPy to create its own [arrays](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html). For this reason, depending on the data type, `values` can either be a `pandas.array` or `numpy.array` object. Therefore, if we need to ensure we get a specific type back, then it is recommended to use the `array` attribute or `to_numpy()` method, respectively, instead of `values`.

Now let's see some examples using these attributes.

### Getting the name of the series
The NumPy array held the name of the data in the `dtype` attribute; here, we can access it directly: 

In [20]:
place.name

'place'

### Getting the data type
A `Series` object holds a single data type. Here it is `'O'` for object.

In [21]:
place.dtype

dtype('O')

### Getting the dimensions of the series
Just as with NumPy, we can use `shape` to get the dimensions as `(rows, columns)`. `Series` objects are a single column, so they only have values for the rows dimension. 

In [22]:
place.shape

(5,)

### Isolating the values from the series
This `Series` object is storing its values as a NumPy array:

In [23]:
place.values

array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
       '42km WNW of Sola, Vanuatu',
       '13km E of Nueva Concepcion, Guatemala',
       '128km SE of Kimbe, Papua New Guinea'], dtype=object)

### 아래 있는 Index는 슬라이드에서 설명을 하시고 실습을 하신다고 하심.

## `Index`
The addition of the `Index` class makes the `Series` class more powerful than a NumPy array. We can get the index from the `index` attribute of a `Series` object:

In [24]:
place #place라는 seriec가 있다.

0          262km NW of Ozernovskiy, Russia
1              25km E of Bitung, Indonesia
2                42km WNW of Sola, Vanuatu
3    13km E of Nueva Concepcion, Guatemala
4      128km SE of Kimbe, Papua New Guinea
Name: place, dtype: object

In [25]:
place_index = place.index
place_index

RangeIndex(start=0, stop=5, step=1)

As with `Series` objects, we can access the underlying data via the `values` attribute. Note that this `Index` object is also built on top of a NumPy array:

In [26]:
place_index.values

array([0, 1, 2, 3, 4])

Here are some commonly used attributes with `Index` objects:

|Attribute | Returns |
| --- | --- |
| `name` | The name of the `Index` object |
| `dtype` | The data type of the `Index` object |
| `shape` | Dimensions of the `Index` object |
| `values` | The data in the `Index` object |
| `is_unique` | Check if the `Index` object has all unique values |

We can check the type of the underlying data, just like with a `Series` object:

In [27]:
place_index.dtype

dtype('int64')

* dtype은 integer라는 것을 알 수 있다.

Same for the dimensions:

In [28]:
place_index.shape

(5,)

We can check if the values are unique:

In [29]:
place_index.is_unique

True

* place_index.is_unique 이게 unique한 인덱스를 가지고 인냐를 확인하는 것이다.

With NumPy we can perform arithmetic operations element-wise between arrays:

* 아래 보이는 코드는 numpy로 만들어 진 코드로 더하기를 할 때는 값을 위치 기반해서 사용을 한다.

In [30]:
np.array([1, 1, 1]) + np.array([-1, 0, 1])

array([0, 1, 2])

Pandas supports this as well, and the index determines how element-wise operations are performed. With addition, only the matching indices are summed:

In [31]:
numbers = np.linspace(0, 10, num=5) # makes numpy array([0, 2.5, 5, 7.5, 10])
x = pd.Series(numbers) # index is [0, 1, 2, 3, 4]
y = pd.Series(numbers, index=pd.Index([1, 2, 3, 4, 5]))
x + y

0     NaN
1     2.5
2     7.5
3    12.5
4    17.5
5     NaN
dtype: float64

* 근데, 여기 pandas에서는 더하기를 할때, index를 기반으로 해서 사용하는 것을 볼 수 있다.

We aren't limited to the integer indices of list-like structures, and we can label our rows. The labels can be altered at any time and be things like dates or even another column. In chapter 3, we will discuss how to perform some operations on the index in order to change it. Then, in chapter 4, we will use the index for operations merging data and aggregating it.

## `DataFrame`
Having a `Series` object for each column is an improvement over the NumPy representation; however, we still have the same problem when wanting to sort based on a value or grab an entire row out. The `DataFrame` gives us a representation of a table formed from many `Series` objects that form the columns and a shared `Index` object that labels the rows. We can create a `DataFrame` object from either of the NumPy representations we were working with earlier (we could also make a `Series` object for each column, but there is no need to do so):

In [32]:
array_dict

{'alert': array(['green', 'green', 'green', 'green', 'green'], dtype='<U5'),
 'mag': array([6.7, 5.2, 5.7, 5.7, 5.6]),
 'magType': array(['mww', 'mww', 'mww', 'mww', 'mww'], dtype='<U3'),
 'place': array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
        '42km WNW of Sola, Vanuatu',
        '13km E of Nueva Concepcion, Guatemala',
        '128km SE of Kimbe, Papua New Guinea'], dtype='<U37'),
 'time': array(['2018-10-13 11:10:23.560', '2018-10-13 04:34:15.580',
        '2018-10-13 00:13:46.220', '2018-10-12 21:09:49.240',
        '2018-10-12 02:52:03.620'], dtype='<U23'),
 'tsunami': array([1, 0, 0, 0, 1])}

* 이 해당 dictionary를 DataFrame형식으로 읽어 오게 된다.

In [33]:
df = pd.DataFrame(array_dict) 

# this will also work with the first representation
# df = pd.DataFrame(data)

df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia",mww,6.7,green,1
1,2018-10-13 04:34:15.580,"25km E of Bitung, Indonesia",mww,5.2,green,0
2,2018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu",mww,5.7,green,0
3,2018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala",mww,5.7,green,0
4,2018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea",mww,5.6,green,1


* 그럼 dictionary형태로 출력이 되는 것을 볼 수 있다.
* DataFrame은 Seriec를 모아놓은 object라는 것을 알 수 있다.

We can check the type of the underlying data with `dtypes` (note that it is not `dtype` as with `Series` and `Index` objects since each column will have its own data type):

In [34]:
df.dtypes

time        object
place       object
magType     object
mag        float64
alert       object
tsunami      int64
dtype: object

* dtypes을 사용해서 각 object들의 dtypes이 나오는 것을 볼 수 있다.

We can get the underlying data with the `values` attribute. Note that this looks very similar to our initial NumPy representation:

In [35]:
df.values

array([['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
        'mww', 6.7, 'green', 1],
       ['2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww',
        5.2, 'green', 0],
       ['2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww',
        5.7, 'green', 0],
       ['2018-10-12 21:09:49.240',
        '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0],
       ['2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea',
        'mww', 5.6, 'green', 1]], dtype=object)

We can isolate the columns with the `columns` attribute. Notice that the columns are actually an `Index` object just on a different axis (columns are the horizontal index while rows are the vertical index).

In [36]:
df.columns

Index(['time', 'place', 'magType', 'mag', 'alert', 'tsunami'], dtype='object')

* columns들의 이름이 출력되는 것을 볼 수 있다.

Here are some commonly used attributes:

|Attribute | Returns |
| --- | --- |
| `dtypes` | The data types of each column |
| `shape` | Dimensions of the `DataFrame` object in a tuple of the form `(number of rows, number of columns)` |
| `index` | The `Index` object along the rows of the `DataFrame` object |
| `columns` | The name of the columns (as an `Index` object) |
| `values` | The data in the `DataFrame` object |
| `empty` | Check if the `DataFrame` object is empty |

The `Index` object along the rows of the dataframe can be accessed via the `index` attribute (just as with `Series` objects):

In [37]:
df.index

RangeIndex(start=0, stop=5, step=1)

* index는 Series의 index와 똑같은 것을 볼 수 있다.

As with both `Series` and `Index` objects, we can get the dimensions of the dataframe with the `shape` attribute. The result is of the form `(nrows, ncols)`. Our dataframe has 5 rows and 6 columns:

In [38]:
df.shape

(5, 6)

* shape는 데이터의 차원을 나타내고 5행 6열이라는 것을 볼 수 있다.

Note that we can also perform arithmetic on dataframes. Pandas will only perform the operation when both the index and column match. Here, we demonstrate addition. Since addition with strings means concatenation, `pandas` concatenated the string columns (`time`, `place`, `magType`, and `alert`) across dataframes. The numeric columns (`mag` and `tsunami`) were summed:

In [39]:
df + df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.5602018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia262km NW of Oze...",mwwmww,13.4,greengreen,2
1,2018-10-13 04:34:15.5802018-10-13 04:34:15.580,"25km E of Bitung, Indonesia25km E of Bitung, I...",mwwmww,10.4,greengreen,0
2,2018-10-13 00:13:46.2202018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu42km WNW of Sola, Van...",mwwmww,11.4,greengreen,0
3,2018-10-12 21:09:49.2402018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala13km E of...",mwwmww,11.4,greengreen,0
4,2018-10-12 02:52:03.6202018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea128km SE of...",mwwmww,11.2,greengreen,2


* DataFrame에서도 덧셈을 할 수 있는 것을 볼 수 있다.
* 더해주게 되면, string과 같은 경우는 같은 거를 두번 나오는 것을 볼 수 있다.
* 근데 숫자 같은 경우는 더해주는 것을 볼 수 있다.

<hr>
<div>
    <a href="../ch_01/introduction_to_data_analysis.ipynb">
        <button style="float: left;">&#8592; Chapter 1</button>
    </a>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>